<a href="https://colab.research.google.com/github/brianellis1997/Sarcasm_Detection/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
!git clone https://github.com/brianellis1997/Sarcasm_Detection.git

Cloning into 'Sarcasm_Detection'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 224 (delta 127), reused 115 (delta 62), pack-reused 3
Receiving objects: 100% (224/224), 4.42 MiB | 11.61 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_bal = pd.read_csv('/content/drive/MyDrive/Sarcasm_Data/Train_Balanced.csv')   # Make sure path is correct in your google drive
train_bal.head()

,Unnamed: 0,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,522142,0,"I personally wasn't a huge Garrosh fan, I've a...",cromemako83,AskReddit,2,2,0,2015-07-01,2015-07-11 01:55:53,Fuck Vol'jin. Garrosh Hellscream did nothing w...
1,907864,1,you forgot the,_SharkWeek_,AskReddit,1,1,0,2013-03-01,2013-03-14 03:03:46,That's a lie fed to you by the LIEberal media....
2,604170,1,"Nah man, she's clearly an ad carry",jdswift13,leagueoflegends,1,1,0,2015-10-01,2015-10-21 23:22:17,she isnt already?
3,110635,1,This sub in a nutshell.,trickz-M-,GlobalOffensive,1,-1,-1,2016-12-01,2016-12-05 03:50:18,Cloud 9 Qualify! (ONLY C9 FANS ALLOWED(
4,997758,0,Yes... I do.,guriboysf,videos,4,4,0,2010-01-01,2010-01-17 21:32:40,"""so, i hear you have a fat cock."""


In [ ]:
# import numpy as np
# from nltk.tokenize import word_tokenize
# from gensim.models import KeyedVectors
# import nltk
# nltk.download('punkt')

# # Load GloVe word vectors
# glove_file = '/content/drive/MyDrive/Sarcasm_Data/glove.6B.100d.txt'  # Adjust the path to your downloaded GloVe file
# word_vectors = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)

# # Function to obtain word vectors for a sentence
# def get_sentence_embedding(sentence):
#     words = word_tokenize(sentence)  # Tokenize sentence into words
#     word_embeddings = []
#     for word in words:
#         try:
#             word_embeddings.append(word_vectors[word])  # Get GloVe vector for word
#         except KeyError:
#             # Handle out-of-vocabulary words
#             pass
#     if len(word_embeddings) == 0:
#         # Handle case where all words are out-of-vocabulary
#         return np.zeros(word_vectors.vector_size)
#     else:
#         # Aggregate word vectors (take sum)
#         return np.sum(word_embeddings, axis=0)

# # Apply to comment and parent comment
# comment_vectors = train_bal['comment'].apply(get_sentence_embedding)
# parent_comment_vectors = train_bal['parent_comment'].apply(get_sentence_embedding)

# Convert comment_vectors and parent_comment_vectors to NumPy arrays
# comment_vectors_np = np.array(comment_vectors.tolist())
# parent_comment_vectors_np = np.array(parent_comment_vectors.tolist())

# # Save the NumPy arrays to files
# np.save('/content/drive/MyDrive/Sarcasm_Data/comment_vectors.npy', comment_vectors_np)
# np.save('/content/drive/MyDrive/Sarcasm_Data/parent_comment_vectors.npy', parent_comment_vectors_np)

In [ ]:
# Load the saved NumPy arrays
# comment_vectors = np.load('/content/drive/MyDrive/Sarcasm_Data/comment_vectors.npy')
# parent_comment_vectors = np.load('/content/drive/MyDrive/Sarcasm_Data/parent_comment_vectors.npy')

In [ ]:
import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec
word2vec = gensim.models.Word2Vec.load('/content/drive/MyDrive/Sarcasm_Data/word2vec.model')

In [5]:
import re, string, time, nltk
import numpy as np
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords as stopwordprovider
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
y = train_bal['label']
X = train_bal.drop(columns=['label'])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#Preparing train data

In [8]:
X_text = X_train[['comment', 'parent_comment']]
# X_numeric = X_train[['word_count', 'capital_letters_count', 'total_punctuation']]

In [ ]:
# X_numeric.shape[0]

646894

In [9]:
X_text.shape[0]

646894

In [24]:
max_words = 1000  # Maximum number of words to keep based on word frequency
max_seq_length = 100  # Maximum length of sequences

In [25]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, concatenate, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

tokenizer = Tokenizer(num_words=max_words, lower=False)
tokenizer.fit_on_texts(X_text['comment'] + ' ' + X_text['parent_comment'])

***Possibly tokenize differenlty for comment and parent comment****

In [26]:
X_text_comment_seq = tokenizer.texts_to_sequences(X_text['comment'])
X_text_parent_comment_seq = tokenizer.texts_to_sequences(X_text['parent_comment'])

In [27]:
X_text_comment_pad = pad_sequences(X_text_comment_seq, maxlen=max_seq_length)
X_text_parent_comment_pad = pad_sequences(X_text_parent_comment_seq, maxlen=max_seq_length)

In [ ]:
# scaler = StandardScaler()
# X_numeric_scaled = scaler.fit_transform(X_numeric)

#Word2vec model trained on our text data

In [28]:
import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec
word2vec = gensim.models.Word2Vec.load('/content/drive/MyDrive/Sarcasm_Data/word2vec.model')

In [15]:
similar_words = word2vec.wv.most_similar('ok')
similar_words

[('okay', 0.9446449279785156),
 ('OK', 0.9253052473068237),
 ('alright', 0.7982171773910522),
 ('fine', 0.6672182083129883),
 ('perfectly_fine', 0.642020046710968),
 ('cool', 0.5878971815109253),
 ('justified', 0.5626591444015503),
 ('wrong', 0.5318138599395752),
 ('understandable', 0.5288909077644348),
 ('unacceptable', 0.4947114884853363)]

In [29]:
from gensim.models import Word2Vec

word_index = {word: idx for idx, word in enumerate(word2vec.wv.index_to_key, start=1)}


In [30]:
import numpy as np
from keras.layers import Embedding

# Assume word2vec is your pre-trained Word2Vec model
# Assume word_index is a dictionary mapping from word to integer index
# Assume max_words is the size of your vocabulary, and word2vec.vector_size is the size of each word vector

embedding_dim = word2vec.vector_size
max_words = len(word_index) + 1  # Adjust based on whether you start indexing from 0 or 1

# Initialize the embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))

# # Populate the embedding matrix
# for word, i in word_index.items():
#     if i < max_words:
#         if word in word2vec.wv:
#             # Words not found in the embedding index will be all zeros
#             embedding_matrix[i] = word2vec.wv[word]



In [31]:
# Initialize the embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))

# Populate the embedding matrix using the tokenizer's word_index
for word, i in tokenizer.word_index.items():
    if i < max_words:  # Ensure you don't exceed the max_words limit
        if word in word2vec.wv:
            embedding_matrix[i] = word2vec.wv[word]


In [32]:
embedding_dim

150

#Preparing Test Data

In [33]:
X_text_test = X_test[['comment', 'parent_comment']]
# X_numeric_test = X_test[['word_count', 'capital_letters_count', 'total_punctuation']]

In [34]:
X_text_comment_seq_test = tokenizer.texts_to_sequences(X_text_test['comment'])
X_text_parent_comment_seq_test = tokenizer.texts_to_sequences(X_text_test['parent_comment'])

In [35]:
X_text_comment_pad_test = pad_sequences(X_text_comment_seq_test, maxlen=max_seq_length)
X_text_parent_comment_pad_test = pad_sequences(X_text_parent_comment_seq_test, maxlen=max_seq_length)

In [ ]:
# from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, concatenate, Dropout, Bidirectional, LSTM
# from keras.models import Model
# from keras import optimizers
# # from kerastuner import HyperModel

# def build_model1(hp):
#     comment_input = Input(shape=(max_seq_length,), name='comment_input')
#     parent_comment_input = Input(shape=(max_seq_length,), name='parent_comment_input')

#     embedding_layer = Embedding(input_dim=max_words,
#                                 output_dim=embedding_dim,
#                                 weights=[embedding_matrix],
#                                 input_length=max_seq_length,
#                                 trainable=True)

#     comment_embedding = embedding_layer(comment_input)
#     parent_comment_embedding = embedding_layer(parent_comment_input)

#     concatenated_embeddings = concatenate([comment_embedding, parent_comment_embedding], axis=1)

#     rnn_layer = SimpleRNN(units=hp.Int('rnn_units', min_value=64, max_value=256, step=64),
#                           dropout=0.2)(concatenated_embeddings)

#     dense = Dense(hp.Int('dense_units', min_value=64, max_value=128, step=32), activation='relu')(rnn_layer)
#     output = Dense(1, activation='sigmoid')(dense)

#     model = Model(inputs=[comment_input, parent_comment_input], outputs=output)

#     hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
#     model.compile(optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])

#     return model

In [37]:
from keras.models import Model
from keras.layers import Input, Embedding, SimpleRNN, Dense, concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Define input shapes
comment_input = Input(shape=(max_seq_length,), name='comment_input')
parent_comment_input = Input(shape=(max_seq_length,), name='parent_comment_input')

# Define embedding layer
embedding_layer = Embedding(input_dim=max_words,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_seq_length,
                            trainable=True)

# Apply embedding layer to inputs
comment_embedding = embedding_layer(comment_input)
parent_comment_embedding = embedding_layer(parent_comment_input)

# Concatenate embeddings
concatenated_embeddings = concatenate([comment_embedding, parent_comment_embedding], axis=1)

# Define SimpleRNN layer
rnn_layer = SimpleRNN(units=128, dropout=0.2)(concatenated_embeddings)

# Define output layer
output = Dense(1, activation='sigmoid')(rnn_layer)

# Create model
model = Model(inputs=[comment_input, parent_comment_input], outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define model checkpoint callback to save the best model
model_checkpoint = ModelCheckpoint(filepath='best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# Train the model
history = model.fit(
    x={'comment_input': X_text_comment_pad, 'parent_comment_input': X_text_parent_comment_pad},
    y=y_train,
    validation_data=({'comment_input': X_text_comment_pad_test, 'parent_comment_input': X_text_parent_comment_pad_test}, y_test),
    epochs=3,
    batch_size=32,
    callbacks=[model_checkpoint]
)


Epoch 1/3
20216/20216 [==============================] - ETA: 0s - loss: 0.6916 - accuracy: 0.5332

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20216/20216 [==============================] - 3905s 193ms/step - loss: 0.6916 - accuracy: 0.5332 - val_loss: 0.6835 - val_accuracy: 0.5564
Epoch 2/3
20216/20216 [==============================] - 3810s 188ms/step - loss: 0.6857 - accuracy: 0.5501 - val_loss: 0.6831 - val_accuracy: 0.5530
Epoch 3/3
20216/20216 [==============================] - 3982s 197ms/step - loss: 0.6855 - accuracy: 0.5528 - val_loss: 0.6992 - val_accuracy: 0.5037


In [ ]:
# Evaluate the best model on the test data
best_model = load_model('best_model.h5')  # Load the best model saved by the model checkpoint callback
loss, accuracy = best_model.evaluate({'comment_input': X_text_comment_pad_test, 'parent_comment_input': X_text_parent_comment_pad_test}, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import numpy as np
from sklearn.metrics import roc_auc_score

# Output directory name
output_dir = 'model_output/rnn'

# Training
epochs = 16
batch_size = 128

# Word embedding properties
n_dim = 100  # assuming your word embeddings are of dimension 100
max_review_length = 100  # adjust according to your requirements
pad_type = trunc_type = 'pre'
drop_embed = 0.2

# RNN layer architecture
n_rnn = 256
drop_rnn = 0.2

# Load word embeddings
word2vec = gensim.models.Word2Vec.load('/content/drive/MyDrive/Sarcasm_Data/word2vec.model')

# Function to convert text to word embeddings
def text_to_embeddings(text):
    embeddings = []
    for word in text.split():
        try:
            embeddings.append(word2vec.wv[word])
        except KeyError:
            # Handle out-of-vocabulary words
            pass
    return np.array(embeddings)

# Preprocess data
train_bal['comment_embedding'] = train_bal['comment'].apply(text_to_embeddings)
train_bal['parent_comment_embedding'] = train_bal['parent_comment'].apply(text_to_embeddings)

# Convert to padded sequences
x_train = pad_sequences(train_bal['comment_embedding'], maxlen=max_review_length, padding=pad_type, truncating=trunc_type, dtype='float32')
x_valid = pad_sequences(train_bal['parent_comment_embedding'], maxlen=max_review_length, padding=pad_type, truncating=trunc_type, dtype='float32')
y_train = train_bal['label'].values

# Design neural network architecture
model = Sequential()
model.add(Embedding(len(word2vec.wv), n_dim, input_length=max_review_length, weights=[word2vec.wv.vectors], trainable=False))
model.add(SpatialDropout1D(drop_embed))
model.add(SimpleRNN(n_rnn, dropout=drop_rnn))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

# Define model checkpoint
model_checkpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")

# Train the model
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2, callbacks=[model_checkpoint])


In [ ]:
# Evaluate the model
y_hat = model.predict_proba(x_valid)
roc_auc = roc_auc_score(y_valid, y_hat)
print("ROC AUC:", roc_auc)

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
similar_words = word2vec.wv.most_similar('ok')
similar_words

[('okay', 0.9446449279785156),
 ('OK', 0.9253052473068237),
 ('alright', 0.7982171773910522),
 ('fine', 0.6672182083129883),
 ('perfectly_fine', 0.642020046710968),
 ('cool', 0.5878971815109253),
 ('justified', 0.5626591444015503),
 ('wrong', 0.5318138599395752),
 ('understandable', 0.5288909077644348),
 ('unacceptable', 0.4947114884853363)]

In [ ]:
word_index = {word: idx for idx, word in enumerate(word2vec.wv.index_to_key, start=1)}

In [ ]:
print(comment_vectors.shape)
print(parent_comment_vectors.shape)

(808618, 100)
(808618, 100)


In [ ]:
print(comment_vectors[0])
print('-'*100)
print(parent_comment_vectors[0])

[-7.95173943e-01  3.14127636e+00  2.78570318e+00 -5.05811024e+00
 -5.42068005e-01  3.63985920e+00 -1.27075803e+00  1.66128492e+00
  3.84848982e-01 -5.39201796e-01  1.56411278e+00 -4.96732265e-01
  3.14057279e+00 -1.09438193e+00  1.24003977e-01 -4.92372990e+00
  1.95057189e+00  1.45621693e+00 -5.86305523e+00  3.62457490e+00
  2.70052624e+00  2.39906836e+00  1.30075109e+00 -1.94248295e+00
  4.43695879e+00  5.67565024e-01 -4.04613018e+00 -4.17990780e+00
  2.51732898e+00 -2.02238202e+00 -4.93887007e-01  7.83402014e+00
  1.92612505e+00  3.37638760e+00  1.19841099e+00  2.33827353e+00
 -1.06820077e-01  4.66233015e+00  1.24715900e+00 -3.10010982e+00
 -3.65968680e+00 -2.88997602e+00  4.48291922e+00 -5.16998005e+00
 -4.32980001e-01 -1.64014041e-01  3.90807080e+00 -3.62764406e+00
  1.86569667e+00 -1.19882402e+01  2.76568204e-01 -2.21814656e+00
  2.40661955e+00  1.25513506e+01 -2.04660821e+00 -3.07382088e+01
 -9.85192060e-02  5.29608965e-01  1.64236794e+01  8.33390999e+00
 -9.90572035e-01  1.01262

In [ ]:
# Add the embeddings to train_bal DataFrame
train_bal['comment_embedding'] = comment_vectors.tolist()
train_bal['parent_comment_embedding'] = parent_comment_vectors.tolist()

# Split the data into features and target
X = train_bal[['comment_embedding']]  # Features
y = train_bal['label']  # Target

# Convert lists to numpy arrays
X['comment_embedding'] = X['comment_embedding'].apply(np.array)
X['parent_comment_embedding'] = X['parent_comment_embedding'].apply(np.array)

<ipython-input-6-962efed96f6d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['comment_embedding'] = X['comment_embedding'].apply(np.array)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=22)

scaler = StandardScaler()
X_train_scaled = np.vstack(X_train['comment_embedding'].to_numpy())
X_val_scaled = np.vstack(X_val['comment_embedding'].to_numpy())
X_train = scaler.fit_transform(X_train_scaled)
X_val = scaler.transform(np.vstack(X_val['comment_embedding'].to_numpy()))

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

In [ ]:
# # Save X_train, X_val, y_train, and y_val
# np.save('/content/drive/MyDrive/Sarcasm_Data/X_train.npy', X_train)
# np.save('/content/drive/MyDrive/Sarcasm_Data/X_val.npy', X_val)
# np.save('/content/drive/MyDrive/Sarcasm_Data/y_train.npy', y_train)
# np.save('/content/drive/MyDrive/Sarcasm_Data/y_val.npy', y_val)

In [ ]:
# # Load X_train, X_val, y_train, and y_val
# X_train = np.load('/content/drive/MyDrive/Sarcasm_Data/X_train.npy')
# X_val = np.load('/content/drive/MyDrive/Sarcasm_Data/X_val.npy')
# y_train = np.load('/content/drive/MyDrive/Sarcasm_Data/y_train.npy')
# y_val = np.load('/content/drive/MyDrive/Sarcasm_Data/y_val.npy')

# # Load the saved NumPy arrays
# comment_vectors = np.load('/content/drive/MyDrive/Sarcasm_Data/comment_vectors.npy')
# parent_comment_vectors = np.load('/content/drive/MyDrive/Sarcasm_Data/parent_comment_vectors.npy')

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the model
model = Sequential([
    LSTM(units=128, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 66689 (260.50 KB)
Trainable params: 66689 (260.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
20216/20216 [==============================] - 167s 8ms/step - loss: 0.6857 - accuracy: 0.5482 - val_loss: 0.6790 - val_accuracy: 0.5657
Epoch 2/10
20216/20216 [==============================] - 164s 8ms/step - loss: 0.6796 - accuracy: 0.5652 - val_loss: 0.6784 - val_accuracy: 0.5670
Epoch 3/10
20216/20216 [==============================] - 162s 8ms/step - loss: 0.6784 - accuracy: 0.5688 - val_loss: 0.6753 - val_accuracy: 0.5752
Epoch 4/10
20216/20216 [==============================] - 157s 8ms/step - loss: 0.6769 - accuracy: 0.5702 - val_loss: 0.6726 - val_accuracy: 0.5796
Epoch 5/10
20216/20216 [==============================] - 157s 8ms/step - loss: 0.6699 - accuracy: 0.5854 - val_loss: 0.6668 - val_accuracy: 0.5920
Epoch 6/10
20216/20216 [==============================] - 156s 8ms/step - loss: 0.6662 - accuracy: 0.5922 - val_loss: 0.6628 - val_accuracy: 0.5975
Epoch 7/10
20216/20216 [==============================] - 161s 8ms/step - loss: 0.6630 - accuracy: 0.5961 - val_

In [ ]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(X_val, y_val)
print('Validation Loss:', loss)
print('Validation Accuracy:', accuracy)

5054/5054 [==============================] - 28s 5ms/step - loss: 0.6680 - accuracy: 0.5876
Validation Loss: 0.6679925918579102
Validation Accuracy: 0.5876493453979492


In [ ]:
y_val[0]

1

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

# Assume train_bal contains the DataFrame with comment embeddings

# Define function to convert data to sparse matrix
def convert_to_sparse(data):
    sparse_data = [csr_matrix(item) for item in data]
    return sparse_data

# Convert comment embeddings to sparse matrices
train_bal['comment_embedding'] = convert_to_sparse(train_bal['comment_embedding'])

# Split the data into features and target
X = train_bal[['comment_embedding']]  # Features
y = train_bal['label']  # Target

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=22)

# Define function to scale and convert data to sparse matrix
def scale_and_convert_to_sparse(data):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    sparse_data = convert_to_sparse(scaled_data)
    return sparse_data

# Scale and convert comment embeddings to sparse matrices for train and validation sets
X_train = scale_and_convert_to_sparse(X_train['comment_embedding'])
X_val = scale_and_convert_to_sparse(X_val['comment_embedding'])

# Define the model
model = Sequential([
    LSTM(units=128, input_shape=(X_train[0].shape[1],), activation='relu'),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Define early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model on validation data
loss, accuracy = model.evaluate(X_val, y_val)
print('Validation Loss:', loss)
print('Validation Accuracy:', accuracy)